In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
# import torch
from concurrent.futures import ThreadPoolExecutor
import json
import warnings
from transformers import pipeline
warnings.filterwarnings("ignore")

2025-08-03 16:53:55.921065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754240036.115465      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754240036.175983      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/emollm/unlabeled_2k.csv')

# Candidate Models:
1. j-hartmann/emotion-english-distilroberta-base [Emotion-Tuned]
2. claudios/codebert-base (microsoft/codebert-base) [Baseline]
3. SamLowe/roberta-base-go_emotions [Emotion-tuned]
6. bhadresh-savani/distilbert-base-uncased-emotion
7. lzw1008/Emollama-7b

In [ ]:
# Mapping for J-Hartmann model emotions
j_hartmann_mapping = {
    'joy': 'satisfaction',
    'neutral': 'neutral',
    'sadness': 'frustration',
    'anger': 'frustration',
    'surprise': 'satisfaction',
    'fear': 'caution',
    'disgust': 'frustration'
}

# Mapping for GoEmotions model (28 labels) to the core set
go_emotions_mapping = {
    'joy': 'satisfaction',
    'excitement': 'satisfaction',
    'gratitude': 'satisfaction',
    'pride': 'satisfaction',
    'approval': 'satisfaction',
    'anger': 'frustration',
    'sadness': 'frustration',
    'annoyance': 'caution',
    'disappointment': 'frustration',
    'disapproval': 'caution',
    'neutral': 'neutral'
    # Other labels will default to 'neutral' in the processing function.
}

# Mapping for DistilBERT model emotions
distilbert_mapping = {
    'joy': 'satisfaction',
    'love': 'satisfaction',
    'surprise': 'satisfaction',
    'sadness': 'frustration',
    'anger': 'frustration',
    'fear': 'caution',
    'Neutral': 'neutral'
}

In [4]:
def map_to_candidate(model_output, mapping_dict, candidate_labels):
    """Map model output to candidate labels using mapping dictionary"""
    best_match = ("neutral", 0.0)
    for item in model_output:
        orig_label = item['label']
        mapped_label = mapping_dict.get(orig_label, "neutral")
        if mapped_label in candidate_labels and item['score'] > best_match[1]:
            best_match = (mapped_label, item['score'])
    return best_match

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

In [6]:
# Define custom hypothesis template and labels
custom_hypothesis_template = "This commit message expresses the emotion of {}."
candidate_labels = ['satisfaction', 'frustration', 'caution', 'neutral']

In [7]:
# --- Model 1: j-hartmann/emotion-english-distilroberta-base ---
print("--- Model 1: j-hartmann/emotion-english-distilroberta-base [Text-Classification] ---")
# This is actually a text-classification model, not zero-shot :cite[1]:cite[6]
classifier1 = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None)  # Return all scores

# --- Model 2: microsoft/codebert-base ---
print("--- Model 2: microsoft/codebert-base [Zero-Shot] ---")
# CodeBERT supports zero-shot classification :cite[4]:cite[5]
classifier2 = pipeline("zero-shot-classification", 
                      model="microsoft/codebert-base",
                      hypothesis_template=custom_hypothesis_template)

# Model 3: General-Purpose Zero-Shot (NLI Gold Standard)
print("--- Model 3: FacebookAI/roberta-large-mnli ---")
classifier3 = pipeline("zero-shot-classification", 
                       model="FacebookAI/roberta-large-mnli")

# --- Model 3: SamLowe/roberta-base-go_emotions ---
print("--- Model 4: SamLowe/roberta-base-go_emotions [Text-Classification] ---")
# This is multi-label text-classification model :cite[10]
classifier4 = pipeline("text-classification", 
                      model="SamLowe/roberta-base-go_emotions",
                      top_k=None)

--- Model 1: j-hartmann/emotion-english-distilroberta-base [Text-Classification] ---


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# --- Model 5: lzw1008/Emollama-7b [Generative Approach] ---
print("--- Model 5: lzw1008/Emollama-7b [Generative] ---")
# Load generative model (requires GPU) :cite[7]
emollama_tokenizer = AutoTokenizer.from_pretrained("lzw1008/Emollama-7b")
emollama_model = AutoModelForCausalLM.from_pretrained(
    "lzw1008/Emollama-7b",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

def emollama_classifier_batched(texts, candidate_labels):
    """Processes a BATCH of texts using a generative model."""
    
    # Create a batch of prompts
    prompts = [
        f"Human:\nTask: Categorize the developer's emotion in this git commit message."
        f"\nChoose ONLY from: {', '.join(candidate_labels)}."
        f"\nMessage: {text}\nAssistant:\nThe emotion is: "
        for text in texts
    ]
    
    # Batch tokenization
    inputs = emollama_tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(emollama_model.device)
    
    # Batch generation
    outputs = emollama_model.generate(**inputs, max_new_tokens=15)
    responses = emollama_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Batch post-processing
    results = []
    for response in responses:
        prediction = response.split("The emotion is: ")[-1].strip().split('\n')[0].split('.')[0].strip()
        
        matched_label = "neutral" # Default
        for label in candidate_labels:
            if label.lower() in prediction.lower():
                matched_label = label
                break
        results.append({'label': matched_label, 'score': 1.0}) # Simplified output
        
    return results


--- Model 5: lzw1008/Emollama-7b [Generative] ---


tokenizer_config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
def predict_emotion(classifier, text, candidate_labels, model_type, mapping=None):
    if model_type == "zero-shot":
        result = classifier(text, candidate_labels=candidate_labels, multi_label=False)
        return result['labels'][0], result['scores'][0]
    elif model_type == "text-classification":
        results = classifier(text)
        if mapping:
            return map_to_candidate(results[0], mapping, candidate_labels)
        else:
            best_label = "neutral"
            best_score = 0.0
            for item in results[0]:
                if item['label'] in candidate_labels and item['score'] > best_score:
                    best_score = item['score']
                    best_label = item['label']
            return best_label, best_score
    elif model_type == "generative":
        result = classifier([text], candidate_labels)[0]
        return result['labels'][0], result['scores'][0]


In [10]:
# Sample prediction on first message
sample_message = df['message'].iloc[0]
print(f"\nSample message: '{sample_message}'\n")

# Model 1 prediction
label1, score1 = predict_emotion(classifier1, sample_message, candidate_labels, "text-classification")
print(f"Model 1 (DistilRoBERTa): {label1} ({score1:.4f})")

# Model 2 prediction
label2, score2 = predict_emotion(classifier2, sample_message, candidate_labels, "zero-shot")
print(f"Model 2 (CodeBERT): {label2} ({score2:.4f})")

# Model 3 prediction
label3, score3 = predict_emotion(classifier3, sample_message, candidate_labels, "zero-shot")
print(f"Model 3 (roberta-large-mnli): {label3} ({score3:.4f})")

# Model 4 prediction
label4, score4 = predict_emotion(classifier4, sample_message, candidate_labels, "text-classification")
print(f"Model 4 (GoEmotions): {label4} ({score4:.4f})")

# Model 5 prediction
output5 = emollama_classifier_batched([sample_message], candidate_labels)[0]
label5, score5 = output5['label'], output5['score'] # Use singular 'label' and 'score'
print(f"Model 5 (Emollama): {label5} ({score5:.4f})")



Sample message: 'roll swiftshader afddeccdfeaa commits git log afddeccdfeaa firstparent format'ad ae s' produce msvc error when virtual class misses virtual destructor created with gclient setdep r swiftshadercdfeaa if this roll has caused a breakage, revert this cl and stop the roller using the controls here please cc on the revert to ensure that a human is aware of the problem. to report a problem with the autoroller itself, please file a bug documentation for the autoroller is here cqincludetrybots luci.chromium.luci.chromium.luci.chromium.luci.chromium.luci.chromium.luci.chromium.swangletryluci.chromium.swangletry bug none tbr'



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model 1 (DistilRoBERTa): neutral (0.7225)
Model 2 (CodeBERT): frustration (0.2502)
Model 3 (roberta-large-mnli): satisfaction (0.3273)
Model 4 (GoEmotions): neutral (0.9569)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model 5 (Emollama): neutral (1.0000)


In [11]:
del classifier1
del classifier2
del classifier3
del classifier4

In [12]:
model_configs = [
    {
        'name': 'distilroberta', 
        'model_id': 'j-hartmann/emotion-english-distilroberta-base',
        'pipeline_type': 'text-classification',
        'mapping': j_hartmann_mapping,
        'batch_size': 32
    },
    {
        'name': 'roberta_nli',
        'model_id': 'FacebookAI/roberta-large-mnli',
        'pipeline_type': 'zero-shot-classification',
        'mapping': None,
        'batch_size': 32
    },
    {
        'name': 'codebert', 
        'model_id': 'microsoft/codebert-base',
        'pipeline_type': 'zero-shot-classification',
        'mapping': None,
        'batch_size': 32
    },
    {
        'name': 'go_emotions', 
        'model_id': 'SamLowe/roberta-base-go_emotions',
        'pipeline_type': 'text-classification', 
        'mapping': go_emotions_mapping,
        'batch_size': 32
    }
]


In [13]:
# 2. Initialize results dictionary
results = {config['name']: [] for config in model_configs}
results['emollama'] = []
all_messages = df['message'].tolist()

# 3. Process the classifier models
batch_size = 32
for config in model_configs:
    print(f"--- Loading and Processing Model: {config['name']} ---")
    
    # LOAD: Create the pipeline for the current model only
    classifier = pipeline(
        config['pipeline_type'],
        model=config['model_id'],
        device=0  # Explicitly assign to GPU
    )
    
    # PREDICT: Run this single model over the entire dataset
    for i in tqdm(range(0, len(all_messages), config['batch_size']), desc=config['name']):
        batch_texts = all_messages[i:i+config['batch_size']]
        
        if config['pipeline_type'] == 'text-classification':
            batch_outputs = classifier(batch_texts, top_k=None)
            for output in batch_outputs:
                label, score = map_to_candidate(output, config['mapping'], candidate_labels)
                results[config['name']].append((label, score))
        else: # zero-shot
            batch_outputs = classifier(batch_texts, candidate_labels=candidate_labels, multi_label=False)
            for output in batch_outputs:
                results[config['name']].append((output['labels'][0], output['scores'][0]))

    # UNLOAD: Delete the classifier and clear GPU cache
    del classifier
    torch.cuda.empty_cache()
    print(f"--- Model {config['name']} processed and unloaded. VRAM cleared. ---")



--- Loading and Processing Model: distilroberta ---


Device set to use cuda:0
distilroberta: 100%|██████████| 63/63 [00:10<00:00,  5.96it/s]


--- Model distilroberta processed and unloaded. VRAM cleared. ---
--- Loading and Processing Model: roberta_nli ---


Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
roberta_nli: 100%|██████████| 63/63 [02:36<00:00,  2.49s/it]


--- Model roberta_nli processed and unloaded. VRAM cleared. ---
--- Loading and Processing Model: codebert ---


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
codebert: 100%|██████████| 63/63 [01:16<00:00,  1.22s/it]


--- Model codebert processed and unloaded. VRAM cleared. ---
--- Loading and Processing Model: go_emotions ---


Device set to use cuda:0
go_emotions: 100%|██████████| 63/63 [00:18<00:00,  3.42it/s]

--- Model go_emotions processed and unloaded. VRAM cleared. ---


In [14]:
# 4. Process the generative model with its own optimized batching
print(f"--- Processing Model: emollama ---")
emollama_batch_size = 4 # Keep this small for large models
for i in tqdm(range(0, len(all_messages), emollama_batch_size), desc='emollama'):
    batch_texts = all_messages[i:i+emollama_batch_size]
    batch_outputs = emollama_classifier_batched(batch_texts, candidate_labels)
    for output in batch_outputs:
        results['emollama'].append((output['label'], output['score']))

--- Processing Model: emollama ---


emollama: 100%|██████████| 500/500 [53:41<00:00,  6.44s/it]


In [15]:
del emollama_model
del emollama_tokenizer
torch.cuda.empty_cache()

In [16]:
# --- FINAL DATAFRAME UPDATE ---

# Add results to DataFrame
df['emotion_distilroberta'] = [label for label, _ in results['distilroberta']]
df['score_distilroberta'] = [score for _, score in results['distilroberta']]

df['emotion_codebert'] = [label for label, _ in results['codebert']]
df['score_codebert'] = [score for _, score in results['codebert']]

df['emotion_roberta_nli'] = [label for label, _ in results['roberta_nli']]
df['score_roberta_nli'] = [score for _, score in results['roberta_nli']]

df['emotion_go_emotions'] = [label for label, _ in results['go_emotions']]
df['score_go_emotions'] = [score for _, score in results['go_emotions']]

df['emotion_emollama'] = [label for label, _ in results['emollama']]
df['score_emollama'] = [score for _, score in results['emollama']]

In [17]:
# Print first 5 results
print("\nFirst 5 predictions:")
print(df[['message', 'emotion_distilroberta', 'emotion_codebert', 
          'emotion_roberta_nli', 'emotion_go_emotions', 'emotion_emollama']].head())
df.to_csv('llm_labeled.csv')
print("\nAll classifications complete!")



First 5 predictions:
                                             message emotion_distilroberta  \
0  roll swiftshader afddeccdfeaa commits git log ...               neutral   
1  query tiles add a debug switch to immediately ...               neutral   
2  remove backforwardcache specific ipc and use m...               neutral   
3  add ios to texturelayerskipwaitforactivation c...               neutral   
4  migrate beforeinstallpromptevent to use gc moj...               neutral   

  emotion_codebert emotion_roberta_nli emotion_go_emotions emotion_emollama  
0     satisfaction        satisfaction             neutral          neutral  
1      frustration             caution             neutral          neutral  
2     satisfaction        satisfaction             neutral          neutral  
3     satisfaction         frustration             neutral          neutral  
4      frustration             neutral             neutral          neutral  

All classifications complete!


In [18]:
manual_labeled=pd.read_csv('/kaggle/input/emollm/labeled_2k.csv')

In [19]:
llm_labeled=df

In [20]:
manual_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            2000 non-null   int64  
 1   commit                2000 non-null   object 
 2   author                2000 non-null   object 
 3   date                  2000 non-null   object 
 4   repo                  2000 non-null   object 
 5   project_size          2000 non-null   object 
 6   time_period           2000 non-null   object 
 7   strat_label           2000 non-null   object 
 8   message               2000 non-null   object 
 9   reconciled_emotion    2000 non-null   object 
 10  reconciled_intensity  2000 non-null   float64
dtypes: float64(1), int64(1), object(9)
memory usage: 172.0+ KB


In [21]:
llm_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   commit                 2000 non-null   object 
 1   author                 2000 non-null   object 
 2   date                   2000 non-null   object 
 3   repo                   2000 non-null   object 
 4   project_size           2000 non-null   object 
 5   time_period            2000 non-null   object 
 6   strat_label            2000 non-null   object 
 7   message                2000 non-null   object 
 8   emotion_distilroberta  2000 non-null   object 
 9   score_distilroberta    2000 non-null   float64
 10  emotion_codebert       2000 non-null   object 
 11  score_codebert         2000 non-null   float64
 12  emotion_roberta_nli    2000 non-null   object 
 13  score_roberta_nli      2000 non-null   float64
 14  emotion_go_emotions    2000 non-null   object 
 15  scor

In [22]:
model_columns = {

'distilroberta': 'emotion_distilroberta',

'codebert': 'emotion_codebert',

'roberta_nli': 'emotion_roberta_nli',

'go_emotions': 'emotion_go_emotions',

'emollama': 'emotion_emollama'

}

In [23]:
# ===== MODIFIED EVALUATION SECTION =====
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

# Ensure consistent label casing
manual_labels = manual_labeled['reconciled_emotion'].str.capitalize()
candidate_labels = [label.capitalize() for label in candidate_labels]

# Create label encoder with all possible labels
le = LabelEncoder()
le.fit(candidate_labels + manual_labels.unique().tolist())


# Convert labels to uniform case
for col in model_columns.values():
    df[col] = df[col].str.capitalize()

# Calculate macro F1 for each model
results = {}
for model_name, col in model_columns.items():
    # Encode both true and predicted labels numerically
    y_true = le.transform(manual_labels)
    y_pred = le.transform(df[col])
    
    f1 = f1_score(
        y_true, 
        y_pred,
        average='macro',
        zero_division=0
    )
    results[model_name] = f1
    print(f"Macro F1 for {model_name}: {f1:.4f}")

# Add intensity-weighted F1 score (if helpful)
print("\nIntensity-weighted F1 (higher intensity errors penalized more):")
for model_name, col in model_columns.items():
    y_true = le.transform(manual_labels)
    y_pred = le.transform(df[col])
    
    # Create sample weights based on intensity
    weights = manual_labeled['reconciled_intensity'] / manual_labeled['reconciled_intensity'].max()
    
    f1 = f1_score(
        y_true, 
        y_pred,
        average='macro',
        sample_weight=weights,
        zero_division=0
    )
    print(f"Intensity-weighted F1 for {model_name}: {f1:.4f}")

# Show label distribution for debugging
print("\nLabel distribution in manual labels:")
print(manual_labels.value_counts(normalize=True))

Macro F1 for distilroberta: 0.1886
Macro F1 for codebert: 0.1968
Macro F1 for roberta_nli: 0.2058
Macro F1 for go_emotions: 0.1374
Macro F1 for emollama: 0.1296

Intensity-weighted F1 (higher intensity errors penalized more):
Intensity-weighted F1 for distilroberta: 0.1922
Intensity-weighted F1 for codebert: 0.1910
Intensity-weighted F1 for roberta_nli: 0.2148
Intensity-weighted F1 for go_emotions: 0.1406
Intensity-weighted F1 for emollama: 0.1321

Label distribution in manual labels:
reconciled_emotion
Neutral         0.3460
Satisfaction    0.2685
Frustration     0.2520
Caution         0.1335
Name: proportion, dtype: float64


In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming manual_labeled and llm_labeled DataFrames are already loaded

# # Function to plot emotion and intensity distributions for a given DataFrame
# def plot_emotion_intensity(df, title):
#     fig, axes = plt.subplots(1, 2, figsize=(15, 5))
#     fig.suptitle(title, fontsize=16)

#     sns.countplot(x='emotion', data=df, ax=axes[0], order=df['emotion'].value_counts().index, palette='viridis')
#     axes[0].set_title('Emotion Distribution')
#     axes[0].set_xlabel('Emotion')
#     axes[0].set_ylabel('Count')
#     axes[0].tick_params(axis='x', rotation=45)

#     sns.histplot(df['intensity'], kde=True, ax=axes[1], bins=9, color='orange')  # Use bins=9 for intensity levels 0-8
#     axes[1].set_title('Intensity Distribution')
#     axes[1].set_xlabel('Intensity')
#     axes[1].set_ylabel('Count')

#     plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#     plt.show()

# # Plot for manual_labeled data
# plot_emotion_intensity(manual_labeled, "Manual Labeling - Emotion & Intensity Distribution")

# # Plot for llm_labeled data
# plot_emotion_intensity(llm_labeled, "LLM Labeling - Emotion & Intensity Distribution")



In [25]:
# Combine the datasets
# combined_data = pd.concat([
#     manual_labeled.assign(source='Manual'),
#     llm_labeled.assign(source='LLM')
# ])

# # Get the order of emotions from manual labeling (to maintain consistency)
# emotion_order = manual_labeled['emotion'].value_counts().index.tolist()

# # Create the comparison plot
# plt.figure(figsize=(12, 6))

# # Use a custom color palette with better contrast
# custom_palette = ['#2E86C1', '#FF91AF']  # Deep blue and red for better distinction

# # Create the plot with ordered categories
# sns.countplot(
#     data=combined_data,
#     x='emotion',
#     hue='source',
#     order=emotion_order,
#     palette=custom_palette
# )

# # Customize the plot
# plt.title('Emotion Distribution: Manual vs. LLM', pad=20)
# plt.xlabel('Emotion', labelpad=10)
# plt.ylabel('Count', labelpad=10)

# # Rotate labels for better readability
# plt.xticks(rotation=45, ha='right')

# # Adjust legend
# plt.legend(title='Data Source', bbox_to_anchor=(1.05, 1), loc='upper left')

# # Adjust layout to prevent label cutoff
# plt.tight_layout()

# plt.show()

In [26]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Combine datasets for visualization
# combined_df = pd.concat([
#     manual_labeled.assign(source='Manual'),
#     llm_labeled.assign(source='LLM')
# ])

# # Create the figure
# plt.figure(figsize=(12, 6))

# # Plot histogram with KDE and step element
# ax = sns.histplot(
#     data=combined_df, x='intensity', hue='source', kde=True,
#     element="step", bins=9, palette='viridis'
# )

# # Set title and labels
# plt.title('Intensity Distribution: Manual vs. LLM')
# plt.xlabel('Intensity')
# plt.ylabel('Count')

# # **Manually extract legend handles and labels**
# handles, labels = ax.get_legend_handles_labels()

# # If the legend is empty, manually create it
# if not handles:
#     from matplotlib.lines import Line2D
#     colors = sns.color_palette('viridis', n_colors=2)
#     handles = [Line2D([0], [0], color=colors[0], lw=2, label='Manual'),
#                Line2D([0], [0], color=colors[1], lw=2, label='LLM')]

# # Set the legend
# plt.legend(handles=handles, title="Source", loc='upper right')

# # Show the plot
# plt.show()


In [27]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Load or assume the data is already in these DataFrames
# # manual_labeled = pd.read_csv("manual_labeled.csv")
# # llm_labeled = pd.read_csv("llm_labeled.csv")

# # Merge both DataFrames on a common identifier (assuming 'id' exists)
# merged_df = manual_labeled.merge(llm_labeled, on='message', suffixes=('_manual', '_llm'))

# # Emotion comparison
# emotion_comparison = pd.crosstab(merged_df['emotion_manual'], merged_df['emotion_llm'])

# plt.figure(figsize=(10, 6))
# sns.heatmap(emotion_comparison, annot=True, cmap='Blues', fmt='d')
# plt.title("Comparison of Emotion Labels: Manual vs. LLM")
# plt.xlabel("LLM Predicted Emotion")
# plt.ylabel("Manually Labeled Emotion")
# plt.show()

# # Intensity comparison
# intensity_comparison = pd.crosstab(merged_df['intensity_manual'], merged_df['intensity_llm'])

# plt.figure(figsize=(8, 6))
# sns.heatmap(intensity_comparison, annot=True, cmap='Oranges', fmt='d')
# plt.title("Comparison of Intensity Labels: Manual vs. LLM")
# plt.xlabel("LLM Predicted Intensity")
# plt.ylabel("Manually Labeled Intensity")
# plt.show()

# # Display misclassified examples if any
# misclassified = merged_df[
#     (merged_df['emotion_manual'] != merged_df['emotion_llm']) |
#     (merged_df['intensity_manual'] != merged_df['intensity_llm'])
# ]

# if not misclassified.empty:
#     print("Misclassified Examples:")
#     print(misclassified[['message', 'emotion_manual', 'emotion_llm', 'intensity_manual', 'intensity_llm']].head(10))
